In [1]:
import os

In [2]:
%pwd

'c:\\Data D\\Eight\\Projects\\Text Summarizer\\Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Data D\\Eight\\Projects\\Text Summarizer\\Text-Summarizer'

In [ ]:
# Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    """
    Data Ingestion configuration
    """
    root_dir: Path 
    source_URL: str 
    local_data_file: Path 
    unzip_dir: Path 

In [40]:
from src.TextSummarizer.constants import *
from src.TextSummarizer.utils.common import read_yaml, create_directories
from src.TextSummarizer.logging import logger

In [41]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath: str = CONFIG_FILE_PATH,
            params_filepath: str = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

        logger.info(f"Data Ingestion config: {data_ingestion_config.root_dir} , {data_ingestion_config.source_URL} , {data_ingestion_config.local_data_file} , {data_ingestion_config.unzip_dir}")
        
        return data_ingestion_config



In [42]:
import os
import urllib.request as request
import zipfile
from src.TextSummarizer.logging import logger
from src.TextSummarizer.utils.common import get_size

In [47]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        """
        Download file from URL and save it locally
        """

        logger.info(f"Downloading file from {self.config.source_URL} to {self.config.local_data_file}")
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}") 


    def extract_zip_file(self):
        """
        Extract zip file to the specified directory
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted files to: {unzip_path}")

In [49]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e 

[2025-04-04 19:16:56,903 : INFO: common: YAML file config\config.yaml loaded successfully.]
[2025-04-04 19:16:56,903 : INFO: common: YAML file params.yaml loaded successfully.]
[2025-04-04 19:16:56,903 : INFO: common: Directory artifacts created successfully.]
[2025-04-04 19:16:56,903 : INFO: common: createrd directory at artifacts]
[2025-04-04 19:16:56,903 : INFO: common: Directory artifacts/data_ingestion created successfully.]
[2025-04-04 19:16:56,903 : INFO: common: createrd directory at artifacts/data_ingestion]
[2025-04-04 19:16:56,919 : INFO: 3601515340: Data Ingestion config: artifacts\data_ingestion , https://github.com/janakajmera/Text-Summarizer/blob/main/research/summarizer-data.zip , artifacts\data_ingestion\data.zip , artifacts\data_ingestion]
[2025-04-04 19:16:56,919 : INFO: 2415286206: Downloading file from https://github.com/janakajmera/Text-Summarizer/blob/main/research/summarizer-data.zip to artifacts\data_ingestion\data.zip]
[2025-04-04 19:16:56,919 : INFO: 24152862